<a href="https://colab.research.google.com/github/chaitanyap1/Datascience-projects/blob/master/janatha_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Load necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
from collections import Counter
#scikit learn  library
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
#Use LogisticRegression for classifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#oversampling
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
dataset = pd.read_csv("/content/drive/My Drive/datasets/train_8wry4cB.csv") 
dataset_test = pd.read_csv("/content/drive/My Drive/datasets/test_Yix80N0.csv") 

In [4]:
dataset.shape

(10500, 5)

In [5]:
dataset_test.shape

(4500, 4)

In [6]:
dataset.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,female
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male


In [0]:
def extract_product(str):
    if ";"  in str:
        prd_lst = str.split(";")
        count_item = len(prd_lst)
        first_lv1 = prd_lst[0].split("/")[0]
        first_lv2 = prd_lst[0].split("/")[1]
        lv1_lst =[]
        lv2_lst =[]
        for item in prd_lst:
            lv1_lst.append(item.split("/")[0])
            lv2_lst.append(item.split("/")[1])
        unique_lv1 = len(set(lv1_lst))
        unique_lv2 = len(set(lv2_lst))
        most_freq_lv1 =  max(lv1_lst, key=Counter(lv1_lst).get)        
    else:
        lv_lst = str.split("/")
        first_lv1 = lv_lst[0]
        first_lv2 = lv_lst[1]
        count_item = 1
        unique_lv1 = 1
        unique_lv2 = 1
        most_freq_lv1 = first_lv1
    return (count_item,first_lv1,first_lv2,unique_lv1,unique_lv2,most_freq_lv1)    

In [0]:
new_col = ('NumProduct','FirstA','FirstB','UniqueA','UniqueB','MostA')      
new_col_lst = dataset['ProductList'].apply(lambda x: extract_product(x))    
new_col_df = pd.DataFrame(new_col_lst.tolist(),columns =new_col)

In [0]:
data = pd.concat([dataset, new_col_df], axis=1)

In [0]:
# Time feature extraction
data['startTime'] = pd.to_datetime(data['startTime'])
data['endTime'] = pd.to_datetime(data['endTime'])
data['duration'] = data['endTime'] - data['startTime']
data['duration'] = data['duration'].astype('timedelta64[m]')
data['weekday'] = data['startTime'].dt.dayofweek
data['hour_24h'] = data['startTime'].dt.hour

In [0]:
drop_lst = ['session_id', 'startTime', 'endTime', 'ProductList']

In [12]:
data = data.drop(drop_lst,axis =1 )
print(data.head())

   gender  NumProduct  FirstA  FirstB  ...   MostA  duration weekday  hour_24h
0  female           4  A00002  B00003  ...  A00002       1.0       0        18
1    male           7  A00001  B00009  ...  A00001       6.0       1        14
2  female           1  A00002  B00001  ...  A00002       0.0       6        15
3  female           3  A00002  B00004  ...  A00002       3.0       6         2
4    male           2  A00001  B00001  ...  A00001       2.0       2        16

[5 rows x 10 columns]


In [13]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gender      10500 non-null  object 
 1   NumProduct  10500 non-null  int64  
 2   FirstA      10500 non-null  object 
 3   FirstB      10500 non-null  object 
 4   UniqueA     10500 non-null  int64  
 5   UniqueB     10500 non-null  int64  
 6   MostA       10500 non-null  object 
 7   duration    10500 non-null  float64
 8   weekday     10500 non-null  int64  
 9   hour_24h    10500 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 820.4+ KB


In [14]:
data.head()

,gender,NumProduct,FirstA,FirstB,UniqueA,UniqueB,MostA,duration,weekday,hour_24h
0,female,4,A00002,B00003,1,1,A00002,1.0,0,18
1,male,7,A00001,B00009,1,1,A00001,6.0,1,14
2,female,1,A00002,B00001,1,1,A00002,0.0,6,15
3,female,3,A00002,B00004,1,1,A00002,3.0,6,2
4,male,2,A00001,B00001,1,1,A00001,2.0,2,16


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gender      10500 non-null  object 
 1   NumProduct  10500 non-null  int64  
 2   FirstA      10500 non-null  object 
 3   FirstB      10500 non-null  object 
 4   UniqueA     10500 non-null  int64  
 5   UniqueB     10500 non-null  int64  
 6   MostA       10500 non-null  object 
 7   duration    10500 non-null  float64
 8   weekday     10500 non-null  int64  
 9   hour_24h    10500 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 820.4+ KB


In [0]:
dataset['gender'] = dataset['gender'].map({'female':1,'male':0})

In [17]:
dataset.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,1
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,0
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,1
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,1
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,0


In [0]:
cat_cols=data.select_dtypes(['object']).columns

In [19]:
cat_cols[1:,]

Index(['FirstA', 'FirstB', 'MostA'], dtype='object')

In [20]:
for col in cat_cols[1:,]:
    dummy=pd.get_dummies(data[col],drop_first=True,prefix=col)
    data=pd.concat([data,dummy],axis=1)
    del data[col]
    print(col)
del dummy

FirstA
FirstB
MostA


In [21]:
print(data.shape)

(10500, 108)


**test data set data prep**

In [0]:
new_col = ('NumProduct','FirstA','FirstB','UniqueA','UniqueB','MostA')      
new_col_lst = dataset_test['ProductList'].apply(lambda x: extract_product(x))    
new_col_df = pd.DataFrame(new_col_lst.tolist(),columns =new_col)

In [0]:
data = pd.concat([dataset_test, new_col_df], axis=1)

In [0]:
# Time feature extraction
data['startTime'] = pd.to_datetime(data['startTime'])
data['endTime'] = pd.to_datetime(data['endTime'])
data['duration'] = data['endTime'] - data['startTime']
data['duration'] = data['duration'].astype('timedelta64[m]')
data['weekday'] = data['startTime'].dt.dayofweek
data['hour_24h'] = data['startTime'].dt.hour

In [0]:
drop_lst1 = ['session_id', 'startTime', 'endTime', 'ProductList']

In [26]:
data_test = data.drop(drop_lst1,axis =1 )
data_test.head()

,NumProduct,FirstA,FirstB,UniqueA,UniqueB,MostA,duration,weekday,hour_24h
0,1,A00002,B00003,1,1,A00002,0.0,1,13
1,1,A00002,B00005,1,1,A00002,0.0,4,13
2,1,A00002,B00002,1,1,A00002,0.0,6,10
3,4,A00002,B00003,1,1,A00002,3.0,1,20
4,1,A00002,B00001,1,1,A00002,0.0,0,19


In [0]:
cat_cols=data_test.select_dtypes(['object']).columns

In [28]:
cat_cols

Index(['FirstA', 'FirstB', 'MostA'], dtype='object')

In [29]:
for col in cat_cols:
    dummy=pd.get_dummies(data_test[col],drop_first=True,prefix=col)
    data_test=pd.concat([data_test,dummy],axis=1)
    del data_test[col]
    print(col)
del dummy

FirstA
FirstB
MostA


In [30]:
data_test.shape

(4500, 96)

In [31]:
data_test.head()

,NumProduct,UniqueA,UniqueB,duration,weekday,hour_24h,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,FirstB_B00026,...,FirstB_B00044,FirstB_B00045,FirstB_B00046,FirstB_B00047,FirstB_B00048,FirstB_B00049,FirstB_B00050,FirstB_B00051,FirstB_B00053,FirstB_B00054,FirstB_B00055,FirstB_B00056,FirstB_B00057,FirstB_B00058,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00075,FirstB_B00077,FirstB_B00079,FirstB_B00088,FirstB_B00091,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011
0,1,1,1,0.0,1,13,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,1,1,0.0,4,13,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1,1,1,0.0,6,10,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,1,1,3.0,1,20,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,1,1,0.0,0,19,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [32]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 96 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NumProduct     4500 non-null   int64  
 1   UniqueA        4500 non-null   int64  
 2   UniqueB        4500 non-null   int64  
 3   duration       4500 non-null   float64
 4   weekday        4500 non-null   int64  
 5   hour_24h       4500 non-null   int64  
 6   FirstA_A00002  4500 non-null   uint8  
 7   FirstA_A00003  4500 non-null   uint8  
 8   FirstA_A00004  4500 non-null   uint8  
 9   FirstA_A00005  4500 non-null   uint8  
 10  FirstA_A00006  4500 non-null   uint8  
 11  FirstA_A00007  4500 non-null   uint8  
 12  FirstA_A00008  4500 non-null   uint8  
 13  FirstA_A00009  4500 non-null   uint8  
 14  FirstA_A00010  4500 non-null   uint8  
 15  FirstA_A00011  4500 non-null   uint8  
 16  FirstB_B00002  4500 non-null   uint8  
 17  FirstB_B00003  4500 non-null   uint8  
 18  FirstB_B